In [1]:
"""Trains a convolutional neural network on the MNIST dataset, then attacks it with the FGSM attack."""
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
import tensorflow as tf

2024-02-13 07:37:09.335775: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-13 07:37:09.338405: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-13 07:37:09.368331: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-13 07:37:09.368382: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-13 07:37:09.369567: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [3]:
tf.compat.v1.disable_eager_execution()

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
import numpy as np

In [5]:
from art.attacks.evasion import BasicIterativeMethod
from art.estimators.classification import KerasClassifier
from art.utils import load_dataset

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Read MNIST dataset
(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset(str("mnist"))

In [7]:
# Create Keras convolutional neural network - basic architecture from Keras examples
# Source here: https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=x_train.shape[1:]))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))

In [8]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [9]:
classifier = KerasClassifier(model=model, clip_values=(min_, max_))
classifier.fit(x_train, y_train, nb_epochs=5, batch_size=128)

2024-02-13 07:37:22.316677: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-02-13 07:37:22.380005: W tensorflow/c/c_api.cc:305] Operation '{name:'training/Adam/dense/kernel/v/Assign' id:462 op device:{requested: '', assigned: ''} def:{{{node training/Adam/dense/kernel/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training/Adam/dense/kernel/v, training/Adam/dense/kernel/v/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [10]:
# Evaluate the classifier on the test set
preds = np.argmax(classifier.predict(x_test), axis=1)
acc = np.sum(preds == np.argmax(y_test, axis=1)) / y_test.shape[0]
print("\nTest accuracy: %.2f%%" % (acc * 100))

/opt/conda/lib/python3.11/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2024-02-13 07:40:04.890877: W tensorflow/c/c_api.cc:305] Operation '{name:'dense_1/Softmax' id:121 op device:{requested: '', assigned: ''} def:{{{node dense_1/Softmax}} = Softmax[T=DT_FLOAT, _has_manual_control_dependencies=true](dense_1/BiasAdd)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.



Test accuracy: 98.95%


In [17]:
x_test_subset = x_test[:500]
y_test_subset = y_test[:500]

In [18]:
preds = np.argmax(classifier.predict(x_test_subset), axis=1)
acc = np.sum(preds == np.argmax(y_test_subset, axis=1)) / y_test_subset.shape[0]
print("\nTest accuracy: %.2f%%" % (acc * 100))


Test accuracy: 98.60%


In [21]:
bim_attack = BasicIterativeMethod(estimator=classifier)

# Generate adversarial examples with C&W L2 attack
x_test_adv_bim = bim_attack.generate(x=x_test)

PGD - Random Initializations: 100%|██████████| 1/1 [00:01<00:00,  1.30s/it]


In [25]:
# Evaluate the classifier on the adversarial examples
preds_bim = np.argmax(classifier.predict(x_test_adv_bim), axis=1)
acc_bim = np.sum(preds_bim == np.argmax(y_test, axis=1)) / y_test.shape[0]
print("Test accuracy on adversarial samples (BIM): {:.2f}%".format(acc_bim * 100))

Test accuracy on adversarial samples (BIM): 0.72%


In [26]:
from sklearn.metrics import precision_recall_fscore_support
y_test_subset = np.argmax(y_test, axis=1)
precision, recall, fscore, support = precision_recall_fscore_support(y_test_subset, preds_bim)

# Calculate micro-averaged precision, recall, and F1 score
precision_micro, recall_micro, fscore_micro, _ = precision_recall_fscore_support(
    y_test_subset, preds_bim, average='micro')

# Calculate macro-averaged precision, recall, and F1 score
precision_macro, recall_macro, fscore_macro, _ = precision_recall_fscore_support(
   y_test_subset, preds_bim, average='macro')

# Print the micro-averaged values
print(f"Micro-averaged precision: {precision_micro:.2f}")
print(f"Micro-averaged recall: {recall_micro:.2f}")
print(f"Micro-averaged F1 score: {fscore_micro:.2f}")

# Print the macro-averaged values
print(f"Macro-averaged precision: {precision_macro:.2f}")
print(f"Macro-averaged recall: {recall_macro:.2f}")
print(f"Macro-averaged F1 score: {fscore_macro:.2f}")

Micro-averaged precision: 0.01
Micro-averaged recall: 0.01
Micro-averaged F1 score: 0.01
Macro-averaged precision: 0.02
Macro-averaged recall: 0.01
Macro-averaged F1 score: 0.01
